In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

Agrupando os ações por subsetor de atuação

In [2]:
df_hist_cot_norm = pd.read_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_hist_cot_norm.csv")
df_hist_cot_norm["Date"] = pd.to_datetime(df_hist_cot_norm["Date"])
df_hist_cot_norm.set_index("Date", inplace=True)

In [3]:
# Listando os subsetores de mercado
df_subsetores = pd.read_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\info_empresas.csv")[["tickers","subsetor"]]
df_subsetores.set_index("tickers", inplace=True)
lista_subsetores = list(set(df_subsetores["subsetor"]))

In [6]:
for subsetor in lista_subsetores:
    
    df_subsetor = pd.DataFrame(index=df_hist_cot_norm.index)
    
    for ticker in df_hist_cot_norm.columns:
    
        if df_subsetores.loc[ticker]["subsetor"] == subsetor:
            
            df_subsetor[ticker] = df_hist_cot_norm[ticker]
    
    df_subsetor.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\hist_cot_norm_subsetores\\{subsetor}.csv")

Gerando um df com o histórico de variação média de cada subsetor

In [8]:
data_inicial = '2013-01-01'
data_final = '2023-10-01'
datas = pd.date_range(start=data_inicial, end=data_final, freq='D')
df_media_subsetores = pd.DataFrame({'Date': datas}).set_index('Date')

for subsetor in lista_subsetores:

    df_hist_cot_ticker = pd.read_csv(
        f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\hist_cot_norm_subsetores\\{subsetor}.csv", 
        index_col="Date")
    
    for data in df_hist_cot_ticker.index:
        
        soma_linha = 0
        cont_tickers_somados = 0
        
        for ticker in df_hist_cot_ticker.columns:
            
            cond_calc_media = df_hist_cot_ticker.loc[data, ticker] !=0 and not np.isnan(df_hist_cot_ticker.loc[data, ticker])
            if cond_calc_media:
                
                soma_linha += df_hist_cot_ticker.loc[data, ticker]
                cont_tickers_somados += 1
        
        if cont_tickers_somados > 0:
            df_hist_cot_ticker.loc[data, "media"] = soma_linha / cont_tickers_somados 
    
    for data in df_hist_cot_ticker.index:
        df_media_subsetores.loc[data, subsetor] = df_hist_cot_ticker.loc[data, "media"]

Trantando valores faltantes

In [9]:
# Removendo datas sem cotação (finais de semana, feriados, ...)
df_media_subsetores.dropna(inplace=True, how="all")

# Tratando datas anteriores à data de referencia (data inicial de amostragem)
for subsetor in df_media_subsetores.columns:

    data_ref = df_media_subsetores[subsetor].first_valid_index()

    for data in df_media_subsetores.index:

        if data < data_ref:

            df_media_subsetores.loc[data, subsetor] = 0

# Tratando datas com cotações faltantes
for subsetor in df_media_subsetores.columns:
    df_media_subsetores[f"{subsetor}"].ffill(inplace=True)

Exportando os histórios das médias das cotações normalizadas de cada subsetor

In [10]:
df_media_subsetores.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_media_subsetores.csv", index=True)

Decompondo as séries temporais das médias das cotações normalizadas de cada subsetor

In [11]:
df_tend_media_subsetores = pd.DataFrame(index = df_media_subsetores.index)
df_saz_media_subsetores = pd.DataFrame(index = df_media_subsetores.index)
df_resid_media_subsetores = pd.DataFrame(index = df_media_subsetores.index)

for subsetor in df_media_subsetores.columns:

    # 22 dias de cotação por mês / 252 dias de cotação por ano / df_media_segm.shape[0]//2 maior período possível para decomposição da base de dados
    
    df_tend_media_subsetores[f"{subsetor}"] = seasonal_decompose(df_media_subsetores[f"{subsetor}"],
                                                period = 252,
                                                model = "additive").trend
    df_saz_media_subsetores[f"{subsetor}"] = seasonal_decompose(df_media_subsetores[f"{subsetor}"],
                                                period = 252,
                                                model = "additive").seasonal
    df_resid_media_subsetores[f"{subsetor}"] = seasonal_decompose(df_media_subsetores[f"{subsetor}"],
                                                period = 252,
                                                model = "additive").resid

df_tend_media_subsetores.dropna(inplace=True)
df_saz_media_subsetores.dropna(inplace=True)
df_resid_media_subsetores.dropna(inplace=True)

In [12]:
df_tend_media_subsetores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_tend_media_subsetores.csv")
df_saz_media_subsetores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_saz_media_subsetores.csv")
df_resid_media_subsetores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_resid_media_subsetores.csv")

Gerando a matriz de correlação entre os subsetores

In [ ]:
mtx_corr_subsetores = df_media_subsetores.corr()
mtx_corr_subsetores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\mtx_corr_subsetores.csv")

Gerando a matriz de correlação entre as sazonalidades da médias dos subsetores

In [ ]:
mtx_corr_saz_subsetores = df_saz_media_subsetores.corr()
mtx_corr_saz_subsetores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\mtx_corr_saz_subsetores.csv")